In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("tennis.csv")
df.head(8)

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Federer won the Swiss Indoors last week by bea...,https://www.express.co.uk/sport/tennis/1038186...


In [3]:
df = pd.read_csv("trial.csv",encoding="cp850")
df.head()

,article_id,article_text,source
0,1,My name is Yash and I'm in college.,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"A Lion lay asleep in the forest, his great hea...",https://www.tennisworldusa.org/tennis/news/Mar...


In [4]:
df['article_text'][0]

"My name is Yash and I'm in college."

In [5]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [6]:
word_embeddings = {}
f = open('../../../../DATASETS/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

C:\Users\Yash\AppData\Local\Temp/ipykernel_18460/2989591663.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [7]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [8]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [9]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [12]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(100):
#   print("ARTICLE:")
#   print(df['article_text'][i])
#   print('\n')
  print("SUMMARY:")
  print(ranked_sentences[i][1])
  print('\n')

SUMMARY:
"The Lion was much amused to think that a Mouse could ever help him.


SUMMARY:
A timid little Mouse came upon him unexpectedly, and in her fright and haste to get away, ran across the Lion's nose.


SUMMARY:
"Now you see that even a Mouse can help a Lion."


SUMMARY:
But he was generous and finally let the Mouse go.Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's net.


SUMMARY:
The Mouse knew the voice and quickly found the Lion struggling in the net.


SUMMARY:
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.


SUMMARY:
Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.


SUMMARY:
Unable to free himself, he filled the forest with his angry roaring.


SUMMARY:
"Please let me go and some day I will surely repay you.


SUMMARY:
"You laughed when I said I would repay you," said the Mouse.


SUMMARY:
A Lion lay asleep in the for

IndexError: list index out of range

In [11]:
print("ARTCLE LENGTH : ",len(df['article_text'][0]))
print("SUMMARY LENGTH : ",len(ranked_sentences[0][1]))

ARTCLE LENGTH :  35
SUMMARY LENGTH :  68
